# Facebook Data Miner Command-Line Interface

This notebook has basically two main purposes:
1. show-off the CLI by examples,
2. serve as a reference for testing the CLI.

The application has a well-defined CLI. The combinations of all the functions we wanted to expose was far too much to use [Click](https://click.palletsprojects.com/en/7.x/) and its deorators. Instead we went for the simple-stupid [Python Fire](https://google.github.io/python-fire/guide/). I call it simple-stupid, because beside the functions that we wanted to expose and some face classes the only thing we had to do is to pass the application's main entrypoint to Python Fire like this:
```
app = App(DATA_PATH)c
Fire(app, name='Facebook-Data-Miner')
```

Let's set up first the correct working directory. This is needed for the CLI  to work. In a shell environment you usually want to do something like the following in the root folder of this project:
```
export PYTHONPATH="$PWD"
```

In [1]:
import os
BASE_PATH = None

In [2]:
if not BASE_PATH:
    BASE_PATH = os.path.dirname(os.path.abspath(os.getcwd()))
BASE_PATH

'/home/levente/projects/facebook-data-miner'

In [3]:
try:
	os.chdir(BASE_PATH)
	print(f"OK! Changed to: {os.getcwd()} directory.")
except:
	print(f"WARNING! Couldn't change directory. Current is: {os.getcwd()}")

OK! Changed to: /home/levente/projects/facebook-data-miner directory.


## Features of the CLI

Now we can start with the CLI tool. Python Fire takes an object and lets the user to call all of the objects public method (in Python that means: methods without a starting underscore).

In this notebook we will cover the following groups of methods:
- friends,
- conversations,
- analyzer,
- people,
- report,
- plot.

This means you can get detailed information for these groups once you add these keywords after the file's name you are calling: that is `./miner/app.py`. Let's get the help for the main entrypoint (NOTE: Python Fire pipes the output of the `help` into an interactive file reader, thus the output will be cut after one screen).

In [4]:
!./miner/app.py --help

INFO: Showing help with the command 'Facebook-Data-Miner -- --help'.

NAME
    Facebook-Data-Miner - Entrypoint.

SYNOPSIS
    Facebook-Data-Miner COMMAND

DESCRIPTION
    Entrypoint.

COMMANDS
    COMMAND is one of the following:

     analyzer

     conversations
       @param kind: @param channels: @param cols: @param output: @return:

     friends
       Mi van

     people

     plot

     profile_information

     report


### Friends

In [5]:
!./miner/app.py friends --help

INFO: Showing help with the command 'Facebook-Data-Miner friends -- --help'.

NAME
    Facebook-Data-Miner friends - Mi van

SYNOPSIS
    Facebook-Data-Miner friends <flags>

DESCRIPTION
    @param sort: a
    @param dates: b
    @param output: c
    @return: list of friends, sorted by @sort, with dates of making friend if @dates is True,
    saved in a csv or json file if @output is a valid path.

FLAGS
    --sort=SORT
    --dates=DATES
    --output=OUTPUT


As you can see friends does not have further executable commands you can follow it up with. The description should be quite clear about what these params/flags can do. Let's just first use it without flags.

In [6]:
!./miner/app.py friends

timestamp,name
2020-02-06 15:25:00,Guy Fawkes
2020-02-06 15:26:40,Daisy Duck
2020-02-06 15:29:01,Teflon Musk
2020-02-12 17:01:52,Dér Dénes
2020-02-21 14:07:59,Tőke Hal
2020-03-14 21:54:52,Foo Bar
2020-03-15 20:18:28,Szett Droxler
2020-04-09 21:42:05,Donald Duck
2020-05-28 15:41:59,John Doe



In [7]:
!./miner/app.py friends --sort=name

timestamp,name
2020-02-06 15:26:40,Daisy Duck
2020-04-09 21:42:05,Donald Duck
2020-02-12 17:01:52,Dér Dénes
2020-03-14 21:54:52,Foo Bar
2020-02-06 15:25:00,Guy Fawkes
2020-05-28 15:41:59,John Doe
2020-03-15 20:18:28,Szett Droxler
2020-02-06 15:29:01,Teflon Musk
2020-02-21 14:07:59,Tőke Hal



In [8]:
!./miner/app.py friends --sort=name --dates=False

,name
0,Daisy Duck
1,Donald Duck
2,Dér Dénes
3,Foo Bar
4,Guy Fawkes
5,John Doe
6,Szett Droxler
7,Teflon Musk
8,Tőke Hal



Notice that this ouput is formatted as a CSV. You can aslo format it as a `json`. Just pass `json` as the value of the `--output` flag.

In [9]:
!./miner/app.py friends --sort=name --output=json

{"name":{"1581002800000":"Daisy Duck","1586468525000":"Donald Duck","1581526912000":"D\u00e9r D\u00e9nes","1584222892000":"Foo Bar","1581002700000":"Guy Fawkes","1590680519000":"John Doe","1584303508000":"Szett Droxler","1581002941000":"Teflon Musk","1582294079000":"T\u0151ke Hal"}}


We can also write this to an `output` file instead of `stdout`.

In [10]:
!./miner/app.py friends --sort=name --output=$PWD/notebooks/out.csv # TODO

Data was written to /home/levente/projects/facebook-data-miner/notebooks/out.csv


In [11]:
!cat $PWD/notebooks/out.csv

timestamp,name
2020-02-06 15:26:40,Daisy Duck
2020-04-09 21:42:05,Donald Duck
2020-02-12 17:01:52,Dér Dénes
2020-03-14 21:54:52,Foo Bar
2020-02-06 15:25:00,Guy Fawkes
2020-05-28 15:41:59,John Doe
2020-03-15 20:18:28,Szett Droxler
2020-02-06 15:29:01,Teflon Musk
2020-02-21 14:07:59,Tőke Hal


Or you can write it to a json, just add a filename that ends with `.json`.

In [12]:
!./miner/app.py friends --sort=name --output=$PWD/notebooks/out.json # TODO

Data was written to /home/levente/projects/facebook-data-miner/notebooks/out.json


In [13]:
!cat $PWD/notebooks/out.json

{"name":{"1581002800000":"Daisy Duck","1586468525000":"Donald Duck","1581526912000":"D\u00e9r D\u00e9nes","1584222892000":"Foo Bar","1581002700000":"Guy Fawkes","1590680519000":"John Doe","1584303508000":"Szett Droxler","1581002941000":"Teflon Musk","1582294079000":"T\u0151ke Hal"}}

Let's clear up.

In [14]:
!rm $PWD/notebooks/out.csv
!rm $PWD/notebooks/out.json

### Conversations
Conversations is also an interface that points to a single function. So let's dive into it.

In [15]:
!./miner/app.py conversations --help

INFO: Showing help with the command 'Facebook-Data-Miner conversations -- --help'.

NAME
    Facebook-Data-Miner conversations - @param kind: @param channels: @param cols: @param output: @return:

SYNOPSIS
    Facebook-Data-Miner conversations <flags>

DESCRIPTION
    @param kind: @param channels: @param cols: @param output: @return:

FLAGS
    --kind=KIND
    --channels=CHANNELS
    --cols=COLS
    --output=OUTPUT


In [16]:
!./miner/app.py conversations

timestamp_ms,sender_name,content,type,partner,videos,audio_files,photos,gifs,reactions,files
2014-09-24 17:02:08.715,Levente Csőke,are you the real teflon musk?,Generic,Teflon Musk,,,,,,
2014-11-09 19:56:46.047,Levente Csőke,older stuff,Generic,Tőke Hal,,,,,,
2014-11-09 20:13:26.047,Tőke Hal,testing multiple files,Generic,Tőke Hal,,,,,,
2014-11-09 23:13:48.715,Levente Csőke,yo,Generic,Tőke Hal,,,,,,
2014-11-09 23:13:48.715,Levente Csőke,are you the real teflon musk?,Generic,Teflon Musk,,,,,,
2014-11-10 12:20:06.047,Tőke Hal,yo,Generic,Tőke Hal,,,,,,
2014-11-10 12:21:46.047,Tőke Hal,zup,Generic,Tőke Hal,,,,,,
2014-11-10 12:21:46.047,Teflon Musk,no,Generic,Teflon Musk,,,,,,
2014-11-10 12:26:46.047,Teflon Musk,no,Generic,Teflon Musk,,,,,,
2014-11-10 12:30:45.145,Levente Csőke,not much,Generic,Tőke Hal,,,,,,
2014-11-22 02:17:25.145,Levente Csőke,,Generic,Teflon Musk,,,[{'uri': 'messages/inbox/TeflonMusk_fSD454F/photos/index.jpeg'}],,,
2014-12-03 16:07:25.145,Levente Csőke,not,Generic,Tőke 

As you can see calling this node of the interfacer prints out all the data private conversation data we have. We don's see, but the parameter `kind` itnernally defaults to `private`. We can change this to `group`.

In [17]:
!./miner/app.py conversations --kind=group

timestamp_ms,sender_name,content,type,partner,photos,gifs
2011-07-17 15:00:08.580,Donald Duck,test,Generic,"Tőke Hal, Foo Bar, Donald Duck and 2 others",,
2011-07-17 15:00:08.580,Levente Csőke,test,Generic,"Foo Bar, John Doe and Teflon Musk",,
2011-07-17 15:00:13.721,Foo Bar,what do you test,Generic,"Tőke Hal, Foo Bar, Donald Duck and 2 others",,
2011-07-17 15:00:13.721,Foo Bar,what do you test,Generic,"Foo Bar, John Doe and Teflon Musk",,
2011-07-17 15:00:32.011,Tőke Hal,basic group messages,Generic,"Tőke Hal, Foo Bar, Donald Duck and 2 others",,
2011-07-17 15:00:32.012,Dér Dénes,blabla,Generic,"Tőke Hal, Foo Bar, Donald Duck and 2 others",,
2011-07-17 15:00:32.012,Teflon Musk,basic group messages,Generic,"Foo Bar, John Doe and Teflon Musk",,
2011-07-17 15:02:54.237,John Doe,ok,Generic,"Foo Bar, John Doe and Teflon Musk",,
2011-07-17 15:02:54.237,Facebook User,ok,Generic,"Tőke Hal, Foo Bar, Donald Duck and 2 others",,
2018-04-19 12:31:42.152,Levente Csőke,marathon?,Generic,marathon,,


You can also filter the conversations to some specific channels.

In [18]:
!./miner/app.py conversations --channels="Foo Bar,Teflon Musk"

timestamp_ms,sender_name,files,type,photos,content,partner,videos,audio_files,gifs,reactions
2014-09-24 17:02:08.715,Levente Csőke,,Generic,,are you the real teflon musk?,Teflon Musk,,,,
2014-11-09 23:13:48.715,Levente Csőke,,Generic,,are you the real teflon musk?,Teflon Musk,,,,
2014-11-10 12:21:46.047,Teflon Musk,,Generic,,no,Teflon Musk,,,,
2014-11-10 12:26:46.047,Teflon Musk,,Generic,,no,Teflon Musk,,,,
2014-11-22 02:17:25.145,Levente Csőke,,Generic,[{'uri': 'messages/inbox/TeflonMusk_fSD454F/photos/index.jpeg'}],,Teflon Musk,,,,
2014-12-26 20:01:46.047,Teflon Musk,[{'uri': 'messages/inbox/TeflonMusk_fSD454F/files/1810.04805.pdf'}],Generic,,,Teflon Musk,,,,
2020-02-13 06:15:28.715,Levente Csőke,,Generic,,Lorem lorim.. foo bar 😡😡😡,Foo Bar,,,,
2020-02-13 06:15:38.715,Foo Bar,,Generic,,Ut akar ... consequat. oO wow :P xd :D,Foo Bar,,,,"[{'reaction': '❤', 'actor': 'Levente Csőke'}]"
2020-02-14 01:42:08.145,Levente Csőke,,Generic,[{'uri': 'messages/inbox/FooBar_n5fd6gG50h/photos/blueber

You can also filter out some columns by providing column names that you want in the output.

In [19]:
!./miner/app.py conversations --channels="Foo Bar" --cols='sender_name,content'

timestamp_ms,sender_name,content
2020-02-13 06:15:28.715,Levente Csőke,Lorem lorim.. foo bar 😡😡😡
2020-02-13 06:15:38.715,Foo Bar,Ut akar ... consequat. oO wow :P xd :D
2020-02-14 01:42:08.145,Levente Csőke,
2020-02-14 04:28:48.047,Foo Bar,
2020-02-14 12:48:48.047,Levente Csőke,Duis duia .. ! xdddddd :D
2020-02-14 15:35:28.047,Foo Bar,Excepteur...laborum. :D
2020-02-14 18:22:08.145,Levente Csőke,
2020-02-18 00:08:48.047,Levente Csőke,What the hack? xdddddd :D
2020-02-18 08:28:48.145,Foo Bar,
2020-02-26 13:42:08.145,Levente Csőke,
2020-03-09 11:48:48.047,Foo Bar,
2020-04-02 20:08:48.047,Levente Csőke,Whet? Check this! :P
2020-04-25 23:42:08.047,Levente Csőke,
2020-05-03 12:15:28.123,Foo Bar,OUT!
2020-08-08 20:22:08.321,Levente Csőke,OUT! ❤



As of the parameter `output`, it works the same as it works in the case of CLI node `friends`. There are 4 possibilities: format csv and json in file or on stdout.

### Messaging Analyzer
Now we got to the most complex node in this CLI. So let's start with the help message. As you will see this only prints out information on the analyzer function's signature. 

In [20]:
!./miner/app.py analyzer --help

INFO: Showing help with the command 'Facebook-Data-Miner analyzer -- --help'.

NAME
    Facebook-Data-Miner analyzer

SYNOPSIS
    Facebook-Data-Miner analyzer <flags>

FLAGS
    --kind=KIND
    --channels=CHANNELS
    --participants=PARTICIPANTS
    --senders=SENDERS
    --start=START
    --end=END
    --period=PERIOD


The function can have two distinct outcome based on wether you provide any value to the `kind` parameter. 

**If not**, then you will get the rather high level MessagingAnalyzer object's methods, which has functionalities that analyze messages both in private and group channels.

**Otherwise**, if `kind` has the value `private` or `group`, you will get a facade object to the lower-level `{Private|Group}MessagingAnalyzer` and `{Private|Group}ConversationStats` objects. This facade has almost 50 methods, and a lot of them take parameters as well. We will cover all of the functions for the sake of completeness, and also because this notebook is a subject for tests. This will be a really long part of the notebook, but it contains most of the information. However if you want to skip to the [goodies](#report), like reports and plot, go ahead.

Let's go with the first story. No value provided for `kind`.

In [21]:
!./miner/app.py analyzer

NAME
    Facebook-Data-Miner analyzer

SYNOPSIS
    Facebook-Data-Miner analyzer - COMMAND

COMMANDS
    COMMAND is one of the following:

     get_who_i_have_private_convo_with_from_a_group

     how_much_i_speak_in_private_with_group_members

     is_priv_msg_first_then_group

     people_i_have_group_convo_with

     people_i_have_private_convo_with


Let's go over these methods. 

**IMPORTANT NOTE**: as you have seen the `analyzer` function has quite a lot input parameters, which of course can be empty. To tell `Python Fire` that you don't want to fill those values, you have to use a separator. The default is a dash (`-`), but you can change this to almost any character (see [this](https://google.github.io/python-fire/guide/#calling-functions) for reference).

In [22]:
!./miner/app.py analyzer - all_interactions --help # NOTE the single `-` after the analyzer

INFO: Showing help with the command 'Facebook-Data-Miner analyzer - -- --help'.

NAME
    Facebook-Data-Miner analyzer

SYNOPSIS
    Facebook-Data-Miner analyzer - COMMAND

COMMANDS
    COMMAND is one of the following:

     get_who_i_have_private_convo_with_from_a_group

     how_much_i_speak_in_private_with_group_members

     is_priv_msg_first_then_group

     people_i_have_group_convo_with

     people_i_have_private_convo_with


Note also, that you can use `--help` with the functions as well. `Python Fire` will read the signature and the docstring of the function and create a help message from them.

In [23]:
!./miner/app.py analyzer - people_i_have_private_convo_with

Tőke Hal
Foo Bar
Teflon Musk
Benedek Elek


In [24]:
!./miner/app.py analyzer - people_i_have_group_convo_with

Tőke Hal
Levente Csőke
Dér Dénes
Facebook User
Donald Duck
Foo Bar
Teflon Musk
John Doe


In [25]:
# TODO warning because of teflons musk is in the group, but hasnt contributed
!./miner/app.py analyzer - get_who_i_have_private_convo_with_from_a_group --group_name=marathon

Foo Bar
Teflon Musk


In [26]:
# TODO same error
!./miner/app.py analyzer - how_much_i_speak_in_private_with_group_members --group_name=marathon

Foo Bar:     15
Teflon Musk: 6


In [27]:
# TODO same error
!./miner/app.py analyzer - is_priv_msg_first_then_group --name='Foo Bar'

True


#### Private and Group Messaging Analyzer
Both `private` and `group` is created with the very same class, but since the minor differences in the inner structure of two  channels there are some methods that make more sense for one and less for the other one. But we will cover this soon.

Now let's start with a help message.

In [28]:
!./miner/app.py analyzer private --help

INFO: Showing help with the command 'Facebook-Data-Miner analyzer private - -- --help'.

NAME
    Facebook-Data-Miner analyzer private

SYNOPSIS
    Facebook-Data-Miner analyzer private - COMMAND

COMMANDS
    COMMAND is one of the following:

     all_channels
       @param name: a partner name. @return: all channels for this partner (private and groups).

     audios

     average_word_length

     cc

     channels

     contributors
       @return:

     created_by_me

     creator

     end

     files

     get_grouped_time_series_data

     gifs

     group_convo_map

     is_group

     max_channel_size

     mc

     mean_channel_size

     media

     media_mc

     media_message_extractor

     message_language_map
       TODO better error handling TODO better WARNING:polyglot.detect.base:Detector is not able to detect the language reliably. WARNING:polyglot.detect.base:Detector is not able to detect the language reliably. WARNING:polyglot.detect.base:Detector is not able to

And one for `group` as well.

In [29]:
!./miner/app.py analyzer group --help

INFO: Showing help with the command 'Facebook-Data-Miner analyzer group - -- --help'.

NAME
    Facebook-Data-Miner analyzer group

SYNOPSIS
    Facebook-Data-Miner analyzer group - COMMAND

COMMANDS
    COMMAND is one of the following:

     all_channels
       @param name: a partner name. @return: all channels for this partner (private and groups).

     audios

     average_word_length

     cc

     channels

     contributors
       @return:

     created_by_me

     creator

     end

     files

     get_grouped_time_series_data

     gifs

     group_convo_map

     is_group

     max_channel_size

     mc

     mean_channel_size

     media

     media_mc

     media_message_extractor

     message_language_map
       TODO better error handling TODO better WARNING:polyglot.detect.base:Detector is not able to detect the language reliably. WARNING:polyglot.detect.base:Detector is not able to detect the language reliably. WARNING:polyglot.detect.base:Detector is not able to detec

Now as you can see both man pages look the same. 

As described above the facade we get exposes methods from both the higher-level MessagingAnalyzer and the lower-level ConversationStats. So in this section we will dive deep into these functionalities.

In [30]:
!./miner/app.py analyzer private - is_group

False


In [31]:
!./miner/app.py analyzer group - is_group

True


As we anticipated. Now all the channels in the two group. A channel is a conversation on Messenger with somebody or in a group.

In [32]:
!./miner/app.py analyzer private - channels

Teflon Musk
Tőke Hal
Benedek Elek
Foo Bar


In [33]:
!./miner/app.py analyzer group - channels

Tőke Hal, Foo Bar, Donald Duck and 2 others
Foo Bar, John Doe and Teflon Musk
marathon


You can also get the number of these channels.

In [34]:
!./miner/app.py analyzer group - number_of_channels

3


 Now all the participants of these channels.

In [35]:
!./miner/app.py analyzer private - participants

Benedek Elek
Foo Bar
Levente Csőke
Teflon Musk
Tőke Hal


In [36]:
!./miner/app.py analyzer group - participants

Donald Duck
Dér Dénes
Facebook User
Foo Bar
John Doe
Levente Csőke
Teflon Musk
Tőke Hal


These are all the people who are in the above channels. Not everyone of them contributed to the channels tho'. For this to show, we have to filter the group messages into one single channel, where test user `Teflon Musk` have not contributed.

In [37]:
!./miner/app.py analyzer group --channels=marathon - participants

Donald Duck
Foo Bar
Levente Csőke
Teflon Musk


In [38]:
#TODO
!./miner/app.py analyzer group --channels=marathon - contributors

Levente Csőke
Foo Bar
Donald Duck


As you can see the number of contributors is only 3, as opposed to participants, which is 4. Note that you can use the `number_of_contributors` function as well.

Next we have a `group_convo_map` data structure, which holds information on, which people is in which channel. In `private` it's fairly straightforward (if not redundant), as the channel name and the particiapant is exactly the same, but for `group` this can be really useful information.

In [39]:
!./miner/app.py analyzer private - group_convo_map

Tőke Hal:      ["Tőke Hal"]
Levente Csőke: ["Benedek Elek", "Teflon Musk", "Foo Bar", "Tőke Hal"]
Foo Bar:       ["Foo Bar"]
Teflon Musk:   ["Teflon Musk"]
Benedek Elek:  ["Benedek Elek"]


In [40]:
!./miner/app.py analyzer group - group_convo_map

Tőke Hal:      ["Tőke Hal, Foo Bar, Donald Duck and 2 others"]
Levente Csőke: ["Foo Bar, John Doe and Teflon Musk", "Tőke Hal, Foo Bar, Donald Duck and 2 others", "marathon"]
Dér Dénes:     ["Tőke Hal, Foo Bar, Donald Duck and 2 others"]
Facebook User: ["Tőke Hal, Foo Bar, Donald Duck and 2 others"]
Donald Duck:   ["Tőke Hal, Foo Bar, Donald Duck and 2 others", "marathon"]
Foo Bar:       ["Foo Bar, John Doe and Teflon Musk", "Tőke Hal, Foo Bar, Donald Duck and 2 others", "marathon"]
Teflon Musk:   ["Foo Bar, John Doe and Teflon Musk", "marathon"]
John Doe:      ["Foo Bar, John Doe and Teflon Musk"]


Next let's check the number of conversations created by our test user.

In [41]:
!./miner/app.py analyzer private - number_of_convos_created_by_me

4


In [42]:
!./miner/app.py analyzer group - number_of_convos_created_by_me

2


We have the max-,mean-, and min_channel size. Again, for `private` all of this will be 2, but for `groups` it is more interesting.

In [43]:
!./miner/app.py analyzer group - min_channel_size

4


In [44]:
!./miner/app.py analyzer group - mean_channel_size

4.666666666666667


In [45]:
!./miner/app.py analyzer group - max_channel_size

6


We can get all the channels for one conversation partner of ours, which is again a more useful feature for `group` convos.

In [46]:
!./miner/app.py analyzer group - all_channels --name='Foo Bar'

Tőke Hal, Foo Bar, Donald Duck and 2 others
Foo Bar, John Doe and Teflon Musk
marathon


In [47]:
!./miner/app.py analyzer group - all_channels --name='John Doe'

Foo Bar, John Doe and Teflon Musk


Another nice function is the `ranking_by_statistic`. You can rank the participants of the conversations by some statistics.

In [48]:
!./miner/app.py analyzer private - ranking_by_statistic

Foo Bar:      48.38709677419355
Tőke Hal:     22.580645161290324
Teflon Musk:  19.35483870967742
Benedek Elek: 9.67741935483871


In [49]:
!./miner/app.py analyzer group - ranking_by_statistic

Donald Duck:   33.333333333333336
Levente Csőke: 22.22222222222222
Foo Bar:       16.666666666666668
Dér Dénes:     5.555555555555555
Facebook User: 5.555555555555555
John Doe:      5.555555555555555
Teflon Musk:   5.555555555555555
Tőke Hal:      5.555555555555555


Note that this function has some parameters. Let's see the manual for this function.

In [50]:
!./miner/app.py analyzer group - ranking_by_statistic --help

INFO: Showing help with the command 'Facebook-Data-Miner analyzer group - ranking_by_statistic -- --help'.

NAME
    Facebook-Data-Miner analyzer group ranking_by_statistic

SYNOPSIS
    Facebook-Data-Miner analyzer group - ranking_by_statistic <flags>

FLAGS
    --by=BY
    --ranking=RANKING
    --top=TOP


We can change the `by` parameter to word count (wc) or character count (cc)...

In [51]:
!./miner/app.py analyzer private - ranking_by_statistic --by=cc

Benedek Elek: 41.56479217603912
Foo Bar:      34.229828850855746
Teflon Musk:  12.71393643031785
Tőke Hal:     11.491442542787286


Note how this changes the ranking. 

The output shows ranking in percent, but we can change it to absolute count.

In [52]:
!./miner/app.py analyzer private - ranking_by_statistic --by=wc --ranking=count

Foo Bar:      34
Benedek Elek: 32
Teflon Musk:  14
Tőke Hal:     11


It is quite possible that if you want to try this out with your own data, you will have tons of entries here. Change the `top` parameter if you want to limit the number of outputs.

In [53]:
!./miner/app.py analyzer private - ranking_by_statistic --by=text_mc --ranking=count --top=3

Foo Bar:     8
Tőke Hal:    7
Teflon Musk: 4


#### Private and Group ConversationStats

**TL;DR**:
We access this object's methods through the same facade through we access the Analyzer object's methods, although there is quite a difference between the two. 

*A detailed description:*
As the name suggests this class is a container for holding statsictical data/information about converations. The basic concept is that it does not know general conversation metadata, since it is only constructed by the messages and the metadata of unique messages (who sent it, what kind of messages is it, when was it sent). This object is created by `MessagingAnalyzer` class by passing in the DataFrame as input. The DataFrame is created from all the conversations that the analyzer holds (remember you can filter them, down to a single conversation). 

So to sum it up, `MessagingAnalyzer` knows about the channels and all the metadata of the conversations, while `ConversationStats` only knows about the messages themselves.

We expose `ConversationStats`' interesting properties and methods, so let's discover them.

In [54]:
!./miner/app.py analyzer private - creator

You got a warning, because this method only makes sense if there is only one conversation under analysis. So we should filter the private conversations first.

In [55]:
!./miner/app.py analyzer private --channels='Teflon Musk' - creator

Levente Csőke


In [56]:
!./miner/app.py analyzer group --channels=marathon - creator #TODO warn

Levente Csőke


You can get the the timestamp of the first and the last message sent. Remember if you don't filter the the messaging data, you will get the first message ever sent by or to you, and the last message before downloading your Facebook data that was sent by or to you.

In [57]:
!./miner/app.py analyzer group - start

2011-07-17 15:00:08.580000


In [58]:
# TODO why same as non-filtered start
# TODO warn
!./miner/app.py analyzer group --participants="Donald Duck" - start

2011-07-17 15:00:08.580000


In [59]:
!./miner/app.py analyzer private --senders="Teflon Musk" - end

2014-12-26 20:01:46.047000


You can get all the `messages` as well, be it `text` or `media`, but you can also get these separately. Since these are pandas DataFrames, you can pipe them into an output file, just as it was possible with `friends` or `conversations`.

In [60]:
!./miner/app.py analyzer private - messages

timestamp_ms,content
2014-09-24 17:02:08.715,are you the real teflon musk?
2014-11-09 19:56:46.047,older stuff
2014-11-09 20:13:26.047,testing multiple files
2014-11-09 23:13:48.715,yo
2014-11-09 23:13:48.715,are you the real teflon musk?
2014-11-10 12:20:06.047,yo
2014-11-10 12:21:46.047,zup
2014-11-10 12:21:46.047,no
2014-11-10 12:26:46.047,no
2014-11-10 12:30:45.145,not much
2014-12-03 16:07:25.145,not
2018-01-10 09:00:28.715,"yo Legyen az, hogy most megprobalok ekezet nelkul irni. Seems pretty easy. I need some english words in here. Right? A magyar szavak felismereset probalom tesztelni ezzekkel a mondatokkal."
2018-01-10 22:08:26.047,zup
2018-01-10 22:17:25.145,not much
2020-02-13 06:15:28.715,Lorem lorim.. foo bar 😡😡😡
2020-02-13 06:15:38.715,Ut akar ... consequat. oO wow :P xd :D
2020-02-14 12:48:48.047,Duis duia .. ! xdddddd :D
2020-02-14 15:35:28.047,Excepteur...laborum. :D
2020-02-18 00:08:48.047,What the hack? xdddddd :D
2020-04-02 20:08:48.047,Whet? Check this! :P
2020-05-0

In [61]:
!./miner/app.py analyzer group --participants="Donald Duck" - messages

timestamp_ms,content
2011-07-17 15:00:08.580,test
2011-07-17 15:00:13.721,what do you test
2011-07-17 15:00:32.011,basic group messages
2011-07-17 15:00:32.012,blabla
2011-07-17 15:02:54.237,ok
2018-04-19 12:31:42.152,marathon?
2018-04-19 12:32:21.074,yapp yapp :D
2018-04-19 12:32:35.273,You named the group marathon.
2018-04-19 13:35:49.717,i start today
2018-04-19 13:38:02.444,we could go but running is free
2018-04-19 13:38:32.776,hmmm
2018-04-19 14:52:39.709,:D



In [62]:
!./miner/app.py analyzer group --senders="Donald Duck" - messages

timestamp_ms,content
2011-07-17 15:00:08.580,test
2018-04-19 13:35:49.717,i start today
2018-04-19 13:38:02.444,we could go but running is free
2018-04-19 14:52:39.709,:D



See, we can filter for `participants` and for `senders`. 

Filtering for the former means, we want all the messages that was sent in a channel where the subject was a participant. 

Filtering for the latter means we only want the subject's messages.

Now let's get the text and media messages only. # TODOOOOOOOOOOOOO

In [63]:
!./miner/app.py analyzer private --channels="Foo Bar" - text

timestamp_ms,content
2020-02-13 06:15:28.715,Lorem lorim.. foo bar 😡😡😡
2020-02-13 06:15:38.715,Ut akar ... consequat. oO wow :P xd :D
2020-02-14 12:48:48.047,Duis duia .. ! xdddddd :D
2020-02-14 15:35:28.047,Excepteur...laborum. :D
2020-02-18 00:08:48.047,What the hack? xdddddd :D
2020-04-02 20:08:48.047,Whet? Check this! :P
2020-05-03 12:15:28.123,OUT!
2020-08-08 20:22:08.321,OUT! ❤



In [64]:
# TODO error
!./miner/app.py analyzer private --channels="Foo Bar" - media

Traceback (most recent call last):
  File "./miner/app.py", line 114, in <module>
    Fire(app, name='Facebook-Data-Miner')
  File "/home/levente/anaconda3/envs/fb/lib/python3.8/site-packages/fire/core.py", line 138, in Fire
    component_trace = _Fire(component, args, parsed_flag_args, context, name)
  File "/home/levente/anaconda3/envs/fb/lib/python3.8/site-packages/fire/core.py", line 463, in _Fire
    component, remaining_args = _CallAndUpdateTrace(
  File "/home/levente/anaconda3/envs/fb/lib/python3.8/site-packages/fire/core.py", line 672, in _CallAndUpdateTrace
    component = fn(*varargs, **kwargs)
  File "/home/levente/projects/facebook-data-miner/miner/visualizer/adapters.py", line 13, in wrapper
    res = func(*args, **kwargs)
  File "/home/levente/projects/facebook-data-miner/miner/visualizer/adapters.py", line 142, in media
    return self._stats.media
  File "/home/levente/projects/facebook-data-miner/miner/message/conversation_stats.py", line 93, in media
    return self.

In [65]:
# TODO finish this

Note that you can also filter by dates. The input flags are `start`, `end`, `period`.

In [the other notebook](facebook-data-miner.ipynb) this is described as.
> Filter by `start` and `end` is pretty intuitive. You can use both datetime objects and strings (however note that you can only use strings in this format `%Y-%m-%d` as defined in [ISO_8601](https://en.wikipedia.org/wiki/ISO_8601)). Feel free to play areound with these filter parameters.
> Filtering by `period` is less intuitive. `period` in this context means a year, a month, a day, an hour. It is not so flexible, but pretty comfortable to use. You have to use `period` with either `start` or `end`. With `start` it's like the following equation `from start to start+period` and with `end` it's like `from end-period to end`.

In [66]:
# TODO SettingWithCopyWarning warning
!./miner/app.py analyzer private --start="2018-01-01" - messages

/home/levente/anaconda3/envs/fb/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
timestamp_ms,content
2018-01-10 09:00:28.715,"yo Legyen az, hogy most megprobalok ekezet nelkul irni. Seems pretty easy. I need some english words in here. Right? A magyar szavak felismereset probalom tesztelni ezzekkel a mondatokkal."
2018-01-10 22:08:26.047,zup
2018-01-10 22:17:25.145,not much
2020-02-13 06:15:28.715,Lorem lorim.. foo bar 😡😡😡
2020-02-13 06:15:38.715,Ut akar ... consequat. oO wow :P xd :D
2020-02-14 12:48:48.047,Duis duia .. ! xdddddd :D
2020-02-14 15:35:28.047,Excepteur...laborum. :D
2020-02-18 00:08:48.047,What the hack? xdddddd :D
2020-04-02 20:08:48.047,Whet? Check this! :P
2020-05-03 12:15:28.123,OUT!
2020-08-08 20:22:08.321,OUT

In [67]:
!./miner/app.py analyzer private --end="2020-02-15" - messages

/home/levente/anaconda3/envs/fb/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
timestamp_ms,content
2014-09-24 17:02:08.715,are you the real teflon musk?
2014-11-09 19:56:46.047,older stuff
2014-11-09 20:13:26.047,testing multiple files
2014-11-09 23:13:48.715,yo
2014-11-09 23:13:48.715,are you the real teflon musk?
2014-11-10 12:20:06.047,yo
2014-11-10 12:21:46.047,zup
2014-11-10 12:21:46.047,no
2014-11-10 12:26:46.047,no
2014-11-10 12:30:45.145,not much
2014-12-03 16:07:25.145,not
2018-01-10 09:00:28.715,"yo Legyen az, hogy most megprobalok ekezet nelkul irni. Seems pretty easy. I need some english words in here. Right? A magyar szavak felismereset probalom tesztelni ezzekkel a mondatokkal."
2018-01-10 22:08:26.047,zup
2018-01

Write these outputs to file like this.

In [68]:
!./miner/app.py analyzer private - messages --output=$BASE_PATH/out.csv

Data was written to /home/levente/projects/facebook-data-miner/out.csv


In [69]:
!cat $BASE_PATH/out.csv

timestamp_ms,content
2014-09-24 17:02:08.715,are you the real teflon musk?
2014-11-09 19:56:46.047,older stuff
2014-11-09 20:13:26.047,testing multiple files
2014-11-09 23:13:48.715,yo
2014-11-09 23:13:48.715,are you the real teflon musk?
2014-11-10 12:20:06.047,yo
2014-11-10 12:21:46.047,zup
2014-11-10 12:21:46.047,no
2014-11-10 12:26:46.047,no
2014-11-10 12:30:45.145,not much
2014-12-03 16:07:25.145,not
2018-01-10 09:00:28.715,"yo Legyen az, hogy most megprobalok ekezet nelkul irni. Seems pretty easy. I need some english words in here. Right? A magyar szavak felismereset probalom tesztelni ezzekkel a mondatokkal."
2018-01-10 22:08:26.047,zup
2018-01-10 22:17:25.145,not much
2020-02-13 06:15:28.715,Lorem lorim.. foo bar 😡😡😡
2020-02-13 06:15:38.715,Ut akar ... consequat. oO wow :P xd :D
2020-02-14 12:48:48.047,Duis duia .. ! xdddddd :D
2020-02-14 15:35:28.047,Excepteur...laborum. :D
2020-02-18 00:08:48.047,What the hack? xdddddd :D
2020-04-02 20:08:48.047,Whet? Check this! :P
2020-05-0

In [70]:
!rm $BASE_PATH/out.csv # clear-up

Get all the messages that has reaction on it? No problem.

In [71]:
!./miner/app.py analyzer private - reacted_messages

timestamp_ms,sender_name,content,type,partner,videos,audio_files,photos,gifs,reactions,files
2020-02-13 06:15:38.715,Foo Bar,Ut akar ... consequat. oO wow :P xd :D,Generic,Foo Bar,,,,,"[{'reaction': '❤', 'actor': 'Levente Csőke'}]",
2020-02-14 18:22:08.145,Levente Csőke,,Generic,Foo Bar,,,,[{'uri': 'messages/inbox/FooBar_n5fd6gG50h/gifs/97999627_1419172538270405_8596479473619042304_n_2963870430335255.gif'}],"[{'reaction': '😮', 'actor': 'Foo Bar'}]",
2020-02-18 08:28:48.145,Foo Bar,,Generic,Foo Bar,,,,[{'uri': 'messages/inbox/FooBar_n5fd6gG50h/gifs/19349964_1624604560892442_7457726181358436352_n_487109582171361.gif'}],"[{'reaction': '❤', 'actor': 'Levente Csőke'}]",



The facade exposes low-level statistics, like `message`, `word`, `character`, `text message` and `media message` **counts**. Let's see them.

In [72]:
!./miner/app.py analyzer private - mc

31


In [73]:
!./miner/app.py analyzer group - wc

40


In [74]:
# TODO does not work
!./miner/app.py analyzer private --channels="Tőke Hal, Foo Bar, Donald Duck and 2 others" - cc

0


In [75]:
# TODO SettingWithCopyWarning warning 
!./miner/app.py analyzer private --start="2018-08-05" - text_mc

/home/levente/anaconda3/envs/fb/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
8


In [76]:
!./miner/app.py analyzer private - media_mc

9


You can get the number of unique messages or words.

In [77]:
!./miner/app.py analyzer private - unique_mc

17


In [78]:
!./miner/app.py analyzer private - unique_wc

70


Or get the most used messages and words in messenger.

In [79]:
!./miner/app.py analyzer private --senders=me - most_used_msgs

,content
are you the real teflon musk?,2
not much,2
What the hack? xdddddd :D,1
Lorem lorim.. foo bar 😡😡😡,1
yo,1
Duis duia .. ! xdddddd :D,1
not,1
"yo Legyen az, hogy most megprobalok ekezet nelkul irni. Seems pretty easy. I need some english words in here. Right? A magyar szavak felismereset probalom tesztelni ezzekkel a mondatokkal.",1
older stuff,1
OUT! ❤,1
Whet? Check this! :P,1



In [80]:
!./miner/app.py analyzer group --senders=partner - most_used_msgs

,content
basic group messages,2
ok,2
what do you test,2
test,1
:D,1
blabla,1
yapp yapp :D,1
i start today,1
we could go but running is free,1



You can also access all the types of media messages:
- photos,
- videos,
- gifs,
- audios,
- files.

Use any of them in the following format.

In [81]:
!./miner/app.py analyzer private - photos

timestamp_ms,photos
2014-11-22 02:17:25.145,[{'uri': 'messages/inbox/TeflonMusk_fSD454F/photos/index.jpeg'}]
2020-02-14 01:42:08.145,[{'uri': 'messages/inbox/FooBar_n5fd6gG50h/photos/blueberry-5417154_960_720.jpg'}]
2020-02-26 13:42:08.145,[{'uri': 'messages/inbox/FooBar_n5fd6gG50h/photos/apple-5391076_960_720.jpg'}]



Speaking of media, you can also see the percentage of media messages and its opposite, percentage of text messages.

In [82]:
!./miner/app.py analyzer private - percentage_of_text_messages

70.96774193548387


In [83]:
!./miner/app.py analyzer group - percentage_of_media_messages

11.111111111111114


What is your average word length?

In [84]:
!./miner/app.py analyzer group --senders=me - average_word_length

5.25


Ok, we arrived at the last two features. These are rather itneresting.

First let's group the low level-stats by time.

In [85]:
!./miner/app.py analyzer private  - get_grouped_time_series_data --period=y

timestamp,mc,text_mc,media_mc,wc,cc
2014-01-01,13,11,2,25,99
2018-01-01,3,3,0,32,170
2020-01-01,15,8,7,34,140



In [86]:
!./miner/app.py analyzer private  - get_grouped_time_series_data --period=m

timestamp,mc,text_mc,media_mc,wc,cc
2014-09-01,1,1,0,6,24
2014-11-01,10,9,1,18,72
2014-12-01,2,1,1,1,3
2018-01-01,3,3,0,32,170
2020-02-01,10,5,5,27,114
2020-03-01,1,0,1,0,0
2020-04-01,2,1,1,4,17
2020-05-01,1,1,0,1,4
2020-08-01,1,1,0,2,5



In [87]:
!./miner/app.py analyzer private  - get_grouped_time_series_data --period=d

timestamp,mc,text_mc,media_mc,wc,cc
2014-09-24,1,1,0,6,24
2014-11-09,4,4,0,12,56
2014-11-10,5,5,0,6,16
2014-11-22,1,0,1,0,0
2014-12-03,1,1,0,1,3
2014-12-26,1,0,1,0,0
2018-01-10,3,3,0,32,170
2020-02-13,2,2,0,14,51
2020-02-14,5,2,3,8,42
2020-02-18,2,1,1,5,21
2020-02-26,1,0,1,0,0
2020-03-09,1,0,1,0,0
2020-04-02,1,1,0,4,17
2020-04-25,1,0,1,0,0
2020-05-03,1,1,0,1,4
2020-08-08,1,1,0,2,5



In [88]:
!./miner/app.py analyzer private  - get_grouped_time_series_data --period=h

timestamp,mc,text_mc,media_mc,wc,cc
2014-09-24 17:00:00,1,1,0,6,24
2014-11-09 19:00:00,1,1,0,2,10
2014-11-09 20:00:00,1,1,0,3,20
2014-11-09 23:00:00,2,2,0,7,26
2014-11-10 12:00:00,5,5,0,6,16
2014-11-22 02:00:00,1,0,1,0,0
2014-12-03 16:00:00,1,1,0,1,3
2014-12-26 20:00:00,1,0,1,0,0
2018-01-10 09:00:00,1,1,0,29,160
2018-01-10 22:00:00,2,2,0,3,10
2020-02-13 06:00:00,2,2,0,14,51
2020-02-14 01:00:00,1,0,1,0,0
2020-02-14 04:00:00,1,0,1,0,0
2020-02-14 12:00:00,1,1,0,6,20
2020-02-14 15:00:00,1,1,0,2,22
2020-02-14 18:00:00,1,0,1,0,0
2020-02-18 00:00:00,1,1,0,5,21
2020-02-18 08:00:00,1,0,1,0,0
2020-02-26 13:00:00,1,0,1,0,0
2020-03-09 11:00:00,1,0,1,0,0
2020-04-02 20:00:00,1,1,0,4,17
2020-04-25 23:00:00,1,0,1,0,0
2020-05-03 12:00:00,1,1,0,1,4
2020-08-08 20:00:00,1,1,0,2,5



Then, let's examine in which periods were/are you the most active?
 
Here period means something different. Note the pattern.

In [89]:
!./miner/app.py analyzer private  - stat_per_period --period=y

2009: 0
2010: 0
2011: 0
2012: 0
2013: 0
2014: 13
2015: 0
2016: 0
2017: 0
2018: 3
2019: 0
2020: 15


In [90]:
!./miner/app.py analyzer private  - stat_per_period --period=m

january:   3
february:  10
march:     1
april:     2
may:       1
june:      0
july:      0
august:    1
september: 1
october:   0
november:  10
december:  2


In [91]:
!./miner/app.py analyzer private  - stat_per_period --period=d

monday:    6
tuesday:   2
wednesday: 6
thursday:  3
friday:    6
saturday:  3
sunday:    5


In [92]:
!./miner/app.py analyzer private  - stat_per_period --period=h

0:  1
1:  1
2:  1
3:  0
4:  1
5:  0
6:  2
7:  0
8:  1
9:  1
10: 0
11: 1
12: 7
13: 1
14: 0
15: 1
16: 1
17: 1
18: 1
19: 1
20: 4
21: 0
22: 2
23: 3


### People
People in an acstracted class which combines the people from the messaging system and your friends. It is a one-method interface.

In [93]:
!./miner/app.py people

,name,friend,message_dir,media_dir
0,Guy Fawkes,True,,
1,Daisy Duck,True,,
2,Teflon Musk,True,teflonmusk_fsd454f,TeflonMusk_fSD454F
3,Dér Dénes,True,,
4,Tőke Hal,True,tokehal_sdf7fs9d876,
5,Foo Bar,True,foobar_n5fd6gG50h,FooBar_n5fd6gG50h
6,Szett Droxler,True,,
7,Donald Duck,True,,
8,John Doe,True,,
9,Benedek Elek,,benedekelek_s4f65sdg,
10,Levente Csőke,,,
11,Facebook User,,,



You can add an `--output` flag to write this in a file as usual.

### Report
<a id='report'>The</a> `report` node of the interface creates nicely formatted tables. Let's see what's in the box.

In [94]:
!./miner/app.py report

NAME
    Facebook-Data-Miner report

SYNOPSIS
    Facebook-Data-Miner report COMMAND

COMMANDS
    COMMAND is one of the following:

     basic_stats

     stats_per_period


In [95]:
!./miner/app.py report basic_stats

+---------+--------------+---------------+------+-----------+----------------+-------------+
| Message | Text message | Media message | Word | Character | Unique message | Unique word |
+---------+--------------+---------------+------+-----------+----------------+-------------+
|    31   |      22      |       9       |  91  |    409    |       17       |      70     |
+---------+--------------+---------------+------+-----------+----------------+-------------+


We have seent his already, but this output looks more concise and prettier of course.

The following tables would be familiar as well.

In [96]:
!./miner/app.py report stats_per_period --period=y

+-----------+------+------+------+------+------+------+------+------+------+------+------+------+
|           | 2009 | 2010 | 2011 | 2012 | 2013 | 2014 | 2015 | 2016 | 2017 | 2018 | 2019 | 2020 |
+-----------+------+------+------+------+------+------+------+------+------+------+------+------+
|  Message  |  0   |  0   |  0   |  0   |  0   |  13  |  0   |  0   |  0   |  3   |  0   |  15  |
|    Word   |  0   |  0   |  0   |  0   |  0   |  25  |  0   |  0   |  0   |  32  |  0   |  34  |
| Character |  0   |  0   |  0   |  0   |  0   |  99  |  0   |  0   |  0   | 170  |  0   | 140  |
+-----------+------+------+------+------+------+------+------+------+------+------+------+------+


In [97]:
!./miner/app.py report stats_per_period --period=m

+-----------+---------+----------+-------+-------+-----+------+------+--------+-----------+---------+----------+----------+
|           | january | february | march | april | may | june | july | august | september | october | november | december |
+-----------+---------+----------+-------+-------+-----+------+------+--------+-----------+---------+----------+----------+
|  Message  |    3    |    10    |   1   |   2   |  1  |  0   |  0   |   1    |     1     |    0    |    10    |    2     |
|    Word   |    32   |    27    |   0   |   4   |  1  |  0   |  0   |   2    |     6     |    0    |    18    |    1     |
| Character |   170   |   114    |   0   |   17  |  4  |  0   |  0   |   5    |     24    |    0    |    72    |    3     |
+-----------+---------+----------+-------+-------+-----+------+------+--------+-----------+---------+----------+----------+


In [98]:
!./miner/app.py report stats_per_period --period=d

+-----------+--------+---------+-----------+----------+--------+----------+--------+
|           | monday | tuesday | wednesday | thursday | friday | saturday | sunday |
+-----------+--------+---------+-----------+----------+--------+----------+--------+
|  Message  |   6    |    2    |     6     |    3     |   6    |    3     |   5    |
|    Word   |   6    |    5    |     39    |    18    |   8    |    2     |   13   |
| Character |   16   |    21   |    197    |    68    |   42   |    5     |   60   |
+-----------+--------+---------+-----------+----------+--------+----------+--------+


In [99]:
!./miner/app.py report stats_per_period --period=h

+-----------+----+---+---+---+---+---+----+---+---+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|           | 0  | 1 | 2 | 3 | 4 | 5 | 6  | 7 | 8 |  9  | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | 21 | 22 | 23 |
+-----------+----+---+---+---+---+---+----+---+---+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|  Message  | 1  | 1 | 1 | 0 | 1 | 0 | 2  | 0 | 1 |  1  | 0  | 1  | 7  | 1  | 0  | 1  | 1  | 1  | 1  | 1  | 4  | 0  | 2  | 3  |
|    Word   | 5  | 0 | 0 | 0 | 0 | 0 | 14 | 0 | 0 |  29 | 0  | 0  | 13 | 0  | 0  | 2  | 1  | 6  | 0  | 2  | 9  | 0  | 3  | 7  |
| Character | 21 | 0 | 0 | 0 | 0 | 0 | 51 | 0 | 0 | 160 | 0  | 0  | 40 | 0  | 0  | 22 | 3  | 24 | 0  | 10 | 42 | 0  | 10 | 26 |
+-----------+----+---+---+---+---+---+----+---+---+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+


### Plot

We can also create some plots with the plot node. See the possible comamnds you can use below.

**NOTE**: since we are calling shell commands from the terminal (and possibly also because of Python Fire) the plots will not show up. There will be another notebook covering these plots.

In [100]:
!./miner/app.py plot

NAME
    Facebook-Data-Miner plot

SYNOPSIS
    Facebook-Data-Miner plot COMMAND

COMMANDS
    COMMAND is one of the following:

     plot_convo_type_ratio

     plot_msg_type_ratio

     plot_ranking_of_friends_by_stats

     plot_stat_count_over_time_series

     plot_stat_count_per_time_period
